In [1]:
import torch
import torch.nn as nn

In [2]:
from diffusers.configuration_utils import ConfigMixin, register_to_config
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Union

In [3]:
import sys
print(sys.path)
sys.path.append('/home/aiteam/tykim/diffusers_experiments')

['/home/aiteam/tykim/codeclone/codef', '/home/aiteam/miniconda3/envs/test/lib/python310.zip', '/home/aiteam/miniconda3/envs/test/lib/python3.10', '/home/aiteam/miniconda3/envs/test/lib/python3.10/lib-dynload', '', '/home/aiteam/miniconda3/envs/test/lib/python3.10/site-packages']


In [4]:
from controlnetfr import ControlNetModelFR
from diffusers import UNet2DConditionModel, ControlNetModel

unet = UNet2DConditionModel.from_pretrained(
        "runwayml/stable-diffusion-v1-5", subfolder="unet", revision=None, cache_dir='/home/aiteam/tykim/hugging/models'
    )

In [6]:
controlnet = ControlNetModelFR.from_unet(unet)

In [9]:
sample = torch.randn(16, 3, 512, 512)
controlnet.controlnet_cond_embedding(sample).shape

torch.Size([16, 320, 64, 64])

In [12]:
controlnet.fr_in(torch.randn(16, 512)).unsqueeze(-1).unsqueeze(-1).shape

torch.Size([16, 320, 1, 1])

In [80]:
class ControlNetModelFR(ControlNetModel):
    _supports_gradient_checkpointing = True

    @register_to_config
    def __init__(
        self,
        in_channels: int = 4,
        conditioning_channels: int = 3,
        flip_sin_to_cos: bool = True,
        freq_shift: int = 0,
        down_block_types: Tuple[str] = (
            "CrossAttnDownBlock2D",
            "CrossAttnDownBlock2D",
            "CrossAttnDownBlock2D",
            "DownBlock2D",
        ),
        only_cross_attention: Union[bool, Tuple[bool]] = False,
        block_out_channels: Tuple[int] = (320, 640, 1280, 1280),
        layers_per_block: int = 2,
        downsample_padding: int = 1,
        mid_block_scale_factor: float = 1,
        act_fn: str = "silu",
        norm_num_groups: Optional[int] = 32,
        norm_eps: float = 1e-5,
        cross_attention_dim: int = 1280,
        attention_head_dim: Union[int, Tuple[int]] = 8,
        num_attention_heads: Optional[Union[int, Tuple[int]]] = None,
        use_linear_projection: bool = False,
        class_embed_type: Optional[str] = None,
        num_class_embeds: Optional[int] = None,
        upcast_attention: bool = False,
        resnet_time_scale_shift: str = "default",
        projection_class_embeddings_input_dim: Optional[int] = None,
        controlnet_conditioning_channel_order: str = "rgb",
        conditioning_embedding_out_channels: Optional[Tuple[int]] = (16, 32, 96, 256),
        global_pool_conditions: bool = False,
    ):
        super().__init__(*args, **kwargs)

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│         super().__init__(in_channels: int = 4,                                                   │
│                                     ▲                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [79]:
controlnet = ControlNetModel.from_unet(unet)

In [33]:
from diffusers import UNet2DConditionModel, ControlNetModel

unet = UNet2DConditionModel.from_pretrained(
        "runwayml/stable-diffusion-v1-5", subfolder="unet", revision=None, cache_dir='/home/aiteam/tykim/hugging/models'
    )

In [34]:
controlnet = ControlNetModel.from_unet(unet)

In [44]:
controlnet.conv_in(torch.randn(1, 4, 64, 64)).shape

torch.Size([1, 320, 64, 64])

In [56]:
controlnet.controlnet_cond_embedding

ControlNetConditioningEmbedding(
  (conv_in): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (blocks): ModuleList(
    (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): Conv2d(32, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Conv2d(96, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (conv_out): Conv2d(256, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [40]:
controlnet.controlnet_cond_embedding(torch.randn(1, 3, 512, 512)).shape

torch.Size([1, 320, 64, 64])

In [64]:
out = nn.Linear(512, 320)(fr_emb)

In [81]:
out.shape

torch.Size([3, 320, 1, 1])

In [87]:
t = torch.randn(3, 320)
t = t.unsqueeze(-1).unsqueeze(-1)

In [86]:
controlnet.conv_in(torch.randn(1, 4, 64, 64)).shape

torch.Size([1, 320, 64, 64])

In [90]:
(t * controlnet.conv_in(torch.randn(1, 4, 64, 64))).shape

torch.Size([3, 320, 64, 64])

In [ ]:
controlnet.controlnet_cond_embedding()

In [69]:
out = out.reshape(3, 320, 1, 1)a

In [ ]:
# 방법1 ) controlnet의 conditonl 인풋의 결과와 modulation 시키기
# 방법2 ) FiLM
# 방법3) 억지로 맞

In [70]:
out.shape

torch.Size([3, 320, 1, 1])

In [71]:
out2 = controlnet.controlnet_cond_embedding(torch.randn(1, 3, 512, 512)) * out

In [72]:
out2.shape

torch.Size([3, 320, 64, 64])

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch

controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)

In [28]:
batch_size = 3
num_channels_latents = 4 # self.unet.config.in_channels
latent_width = 64 # width // self.vae_scale_factor
latent_height = 64 # height // self.vae_scale_factor
fr_emb = torch.randn(batch_size, 512)
latents = torch.randn(batch_size, num_channels_latents, latent_width, latent_height)


In [29]:
latents.shape

torch.Size([3, 4, 64, 64])

In [31]:
nn.Conv2d(4, 1, kernel_size=3, stride=2, padding=1)(latents).shape

torch.Size([3, 1, 32, 32])

In [57]:
layer1 = nn.Linear(512, 1024)

In [59]:
fr_emb = torch.randn(batch_size, 512)

In [60]:
fr_emb = fr_emb.unsqueeze(1)
out = layer1(fr_emb)
out.shape

torch.Size([3, 1, 1024])

In [61]:
from einops import rearrange

rearrange(out, "b c (h w) -> b c h w", h=32, w=32).shape

torch.Size([3, 1, 32, 32])

In [ ]:
nn.Conv2d(1, )

In [27]:
class FuseLayer(nn.Module):
    def __init__(self, fr_emb_dim, num_channels_latents, latent_width, latent_height):
        super().__init__()
        self.fr_emb_dim = fr_emb_dim
        self.num_channels_latents =  num_channels_latents
        self.latent_width = latent_width 
        self.latent_height = latent_height

        self.pool = nn.AdaptiveAvgPool2d()
        self.layer1 = nn.Conv2d(4, 8, kernel_size=3, stride=2, padding=1)
        self.layer2 = nn.Conv2d(8, 1, kernel_size=3, stride=2, padding=1)
        self.layer = nn.Linear(256 + )

    def forward(self, fr_emb, latents):
        """
        args:
        returns:
            fused_latent : [B, 4, 64, ]

        """
        b = latents.size(0)
        # 4, 64 64 -> 4, 32, 32
        x = self.layer1(latents)
        # 4, 32 32 -> 2, 16, 16
        x = self.layer2(x)

        x = x.reshape(b, 2*16*16)

        concat = torch.cat([x, fr_emb])
        

AttributeError: module 'torch.nn' has no attribute 'Moduel'